In [8]:
 pip install tensorflow 

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os, sys
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [9]:
pip install gym[classic_control]

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install pyglet==1.5.27

  Using cached pyglet-1.5.27-py3-none-any.whl (1.1 MB)
  Attempting uninstall: pyglet
    Found existing installation: pyglet 2.0.4
    Uninstalling pyglet-2.0.4:
      Successfully uninstalled pyglet-2.0.4
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!conda install -c conda-forge jupyterlab
!conda install -c anaconda ipython

Policy gradient methods update the probability distribution of actions so that actions with higher expected reward have a higher probability value for an observed state. We will assume discrete (finite) action space and a stochastic (non-deterministic) policy for this

In [3]:
import sys
import torch  
import gym
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt

# Constants
GAMMA = 0.9

class PolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, learning_rate=3e-4):
        super(PolicyNetwork, self).__init__()

        self.num_actions = num_actions
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, num_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.softmax(self.linear2(x), dim=1)
        return x 
    
    def get_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = self.forward(Variable(state))
        highest_prob_action = np.random.choice(self.num_actions, p=np.squeeze(probs.detach().numpy()))
        log_prob = torch.log(probs.squeeze(0)[highest_prob_action])
        return highest_prob_action, log_prob

In [4]:
def update_policy(policy_network, rewards, log_probs):
    discounted_rewards = []

    for t in range(len(rewards)):
        Gt = 0 
        pw = 0
        for r in rewards[t:]:
            Gt = Gt + GAMMA**pw * r
            pw = pw + 1
        discounted_rewards.append(Gt)
        
    discounted_rewards = torch.tensor(discounted_rewards)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9) # normalize discounted rewards

    policy_gradient = []
    for log_prob, Gt in zip(log_probs, discounted_rewards):
        policy_gradient.append(-log_prob * Gt)
    
    policy_network.optimizer.zero_grad()
    policy_gradient = torch.stack(policy_gradient).sum()
    policy_gradient.backward()
    policy_network.optimizer.step()

In [5]:
def main():
    env = gym.make('CartPole-v1')
    policy_net = PolicyNetwork(env.observation_space.shape[0], env.action_space.n, 128)
    
    max_episode_num = 5000
    max_steps = 10000
    numsteps = []
    avg_numsteps = []
    all_rewards = []

    for episode in range(max_episode_num):
        state = env.reset()
        log_probs = []
        rewards = []

        for steps in range(max_steps):
            env.render()
            action, log_prob = policy_net.get_action(state)
            new_state, reward, done, _ = env.step(action)
            log_probs.append(log_prob)
            rewards.append(reward)

            if done:
                update_policy(policy_net, rewards, log_probs)
                numsteps.append(steps)
                avg_numsteps.append(np.mean(numsteps[-10:]))
                all_rewards.append(np.sum(rewards))
                if episode % 1 == 0:
                    sys.stdout.write("episode: {}, total reward: {}, average_reward: {}, length: {}\n".format(episode, np.round(np.sum(rewards), decimals = 3),  np.round(np.mean(all_rewards[-10:]), decimals = 3), steps))
                break
            
            state = new_state
        
    plt.plot(numsteps)
    plt.plot(avg_numsteps)
    plt.xlabel('Episode')
    plt.show()

In [7]:
main()

2023-03-13 15:58:03.366 python[2864:3913148] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fc20a78acd0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-03-13 15:58:03.366 python[2864:3913148] Warning: Expected min height of view: (<NSButton: 0x7fc207f414c0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-03-13 15:58:03.368 python[2864:3913148] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fc200de5690>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-03-13 15:58:03.369 python[2864:3913148] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fc207f41a90>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


NameError: name 'glPushMatrix' is not defined